<a href="https://colab.research.google.com/github/matheuscz/bigData/blob/master/Trab_Text_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install -qq lightgbm
!pip3 install -qq gensim

In [4]:
import re
import nltk
#nltk.download()

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
portuguese_stemmer=nltk.stem.SnowballStemmer('portuguese')

import lightgbm as lgb
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import  SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import  mean_squared_error
import random
import itertools
import datetime

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse

from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
from unicodedata import normalize
from keras.layers import LSTM

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


## Bibliotecas para o text minig

In [0]:
from gensim.models import KeyedVectors

## Leitura dos dados

In [0]:
andamentos=pd.read_pickle('andamentos_tjsp.pkl')

In [9]:
andamentos.head()

,Processo,data,descricao,timedelta,acao,seq
123557,218a98942855ddf259f6ea0bee343b67,1973-05-17,Distribuição Livre Conforme remessa ao Distrib...,11079.0,Inventário e Partilha,1
128428,4bf8c065bc0f776b3598c3990b9ed8c0,1979-09-10,Distribuído por Direcionamento (movimentação e...,13534.0,Inventário e Partilha,1
97535,a9b501b6b76a7eff441ebf5d9c3ec872,1980-04-30,Distribuído por Direcionamento (movimentação e...,12375.0,Desapropriação,1
131228,2fdc177c5ec4f2e2743ac65e0cefd0e4,1980-11-28,Distribuição Livre Processo Distribuído por So...,8859.0,NaN,1
143392,09e89309dde68184c0c4997f19691668,1990-02-13,Incidente Processual Incidente Processual 583....,30.0,Recuperação judicial e Falência,1


## Treino e teste

Teste será tudo de 2017 para frente, treino é tudo antes de 2017

In [0]:
andamentos.data=pd.to_datetime(andamentos.data, format='%Y%m%d', errors='ignore')

In [11]:
andamentos.data.iloc[0]


Timestamp('1973-05-17 00:00:00')

In [13]:
andamentos['descricao'][233]

'Autos no Prazo Ofício Expedido Ofício - Cartório Registro Imóveis - Averbação de Penhora - Execução Fiscal '

In [14]:
andamentos['acao'].value_counts()

Contratos Bancários                                                            7983
Inventário e Partilha                                                          5549
Recuperação judicial e Falência                                                5295
Liquidação / Cumprimento / Execução                                            4750
Espécies de Contratos                                                          4636
Locação de Imóvel                                                              4607
Indenização por Dano Moral                                                     4073
Despesas Condominiais                                                          3803
Cheque                                                                         3753
Duplicata                                                                      3690
Obrigação de Fazer / Não Fazer                                                 2744
Servidor Público Civil                                                      

In [15]:
import time
s = "01/01/2017 00:00:00"
seconds = datetime.datetime.strptime(s, "%d/%m/%Y  %H:%M:%S")
print(seconds)

2017-01-01 00:00:00


In [0]:
import time
s = "01/01/2017 00:00:00"
limite = datetime.datetime.strptime(s, "%d/%m/%Y  %H:%M:%S")

train=andamentos[(andamentos['data']<limite)]  
test=andamentos[(andamentos['data']>=limite)]  

In [17]:
train.shape, test.shape

((76651, 6), (71772, 6))

## Preprocessamento

1- retira acentos
2- retira tudo que não é letra (como pontuação e números)
3- tudo em minúsculo
4- retira stopwords
5- stemmers (deixar só o radical das palavras)

In [0]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocessamento( descricao, remove_stopwords=True):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove acentuacao
    review_text = remover_acentos(descricao)

    #
    # 2. Remove numeros
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Lower case
    words = review_text.lower().split()
    #
    # 4. Remover stopwords
    if remove_stopwords:
        stops = set(stopwords.words("portuguese"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = portuguese_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))

    # 5. Return a list of words
    return(b)

In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
preprocessamento(andamentos.iloc[177].descricao)

['redistribuica',
 'process',
 'said',
 'redistribuica',
 'process',
 'said',
 'process',
 'redistribu',
 'sortei',
 'conf',
 'desp',
 'mm',
 'juiz',
 'fls']

In [0]:
andamentos_revisados_train=[]
for i in range(train.shape[0]):
    andamentos_revisados_train.append(" ".join(preprocessamento(train.iloc[i].descricao)))

    
andamentos_revisados_test=[]
for i in range(test.shape[0]):
    andamentos_revisados_test.append(" ".join(preprocessamento(test.iloc[i].descricao)))

In [0]:
y_train = train['timedelta']
y_test = test['timedelta']

## TF-IDF

In [0]:
vectorizer = TfidfVectorizer()

vectorizer = vectorizer.fit(andamentos_revisados_train)
train_features = vectorizer.transform(andamentos_revisados_train)
test_features = vectorizer.transform(andamentos_revisados_test)

In [25]:
train_features.shape, test_features.shape

((76651, 19388), (71772, 19388))

In [26]:
print(vectorizer.get_feature_names()[2999])

casareg


## LightGBM + TF-IDF

In [42]:
SEED = 1990

model = lgb.LGBMRegressor(boosting_type='gbdt', 
                          num_leaves=31, 
                          max_depth=-1, 
                          learning_rate=0.01, 
                          n_estimators=1000, 
                          max_bin=255, 
                          subsample_for_bin=50000, 
                          objective=None, 
                          min_split_gain=0, 
                          min_child_weight=3,
                          min_child_samples=10, 
                          subsample=1, 
                          subsample_freq=1, 
                          colsample_bytree=1, 
                          reg_alpha=0.1, 
                          reg_lambda=0, 
                          seed=17,
                          silent=False, 
                          nthread=-1)

model.fit(train_features,y_train,eval_set=[(train_features, y_train),(test_features,y_test)], eval_metric="rmse", early_stopping_rounds=100)

[1]	valid_0's rmse: 122.347	valid_0's l2: 14968.9	valid_1's rmse: 35.0811	valid_1's l2: 1230.68
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 122.117	valid_0's l2: 14912.7	valid_1's rmse: 34.9909	valid_1's l2: 1224.37
[3]	valid_0's rmse: 121.892	valid_0's l2: 14857.6	valid_1's rmse: 34.9058	valid_1's l2: 1218.42
[4]	valid_0's rmse: 121.67	valid_0's l2: 14803.6	valid_1's rmse: 34.8256	valid_1's l2: 1212.82
[5]	valid_0's rmse: 121.453	valid_0's l2: 14750.8	valid_1's rmse: 34.7534	valid_1's l2: 1207.8
[6]	valid_0's rmse: 121.24	valid_0's l2: 14699	valid_1's rmse: 34.6792	valid_1's l2: 1202.65
[7]	valid_0's rmse: 121.029	valid_0's l2: 14648.1	valid_1's rmse: 34.6112	valid_1's l2: 1197.94
[8]	valid_0's rmse: 120.824	valid_0's l2: 14598.3	valid_1's rmse: 34.5519	valid_1's l2: 1193.83
[9]	valid_0's rmse: 120.622	valid_0's l2: 14549.6	valid_1's rmse: 34.4952	valid_1's l2: 1189.92
[10]	valid_0's rmse: 120.423	valid_0's l2: 14501.7	valid_1's rmse: 34.4362	val

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
       importance_type='split', learning_rate=0.01, max_bin=255,
       max_depth=-1, min_child_samples=10, min_child_weight=3,
       min_split_gain=0, n_estimators=1000, n_jobs=-1, nthread=-1,
       num_leaves=31, objective=None, random_state=None, reg_alpha=0.1,
       reg_lambda=0, seed=17, silent=False, subsample=1,
       subsample_for_bin=50000, subsample_freq=1)

## Tokenizer e sequences

In [0]:
max_words = 15000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(andamentos_revisados_train)
sequences_train = tokenizer.texts_to_sequences(andamentos_revisados_train)
sequences_test = tokenizer.texts_to_sequences(andamentos_revisados_test)

In [0]:


data_train = pad_sequences(sequences_train, maxlen=1000)
data_test = pad_sequences(sequences_test, maxlen=1000)


## Rede convolucional

In [29]:
from keras.layers import MaxPooling1D,GlobalMaxPooling1D
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=1000))
model.add(Convolution1D(32,7,activation='relu'))
model.add(MaxPooling1D(5))
model.add(Convolution1D(32,7,activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, kernel_initializer='normal'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         1500000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 994, 32)           22432     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 198, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 192, 32)           7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,529,665
Trainable params: 1,529,665
Non-trainable params: 0
_________________________________________________________________


In [30]:
data_train.shape

(76651, 1000)

In [31]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])
history = model.fit(data_train, y_train,
                    epochs=30,
                    batch_size=128,
                    validation_data=(data_test, y_test))

Train on 76651 samples, validate on 71772 samples
Epoch 1/30
76651/76651 [==============================] - 27s 359us/step - loss: 14926.0278 - mean_squared_error: 14926.0278 - val_loss: 1145.0903 - val_mean_squared_error: 1145.0903
Epoch 2/30
76651/76651 [==============================] - 23s 299us/step - loss: 14691.4861 - mean_squared_error: 14691.4861 - val_loss: 1145.2254 - val_mean_squared_error: 1145.2254
Epoch 3/30
76651/76651 [==============================] - 23s 299us/step - loss: 14504.3655 - mean_squared_error: 14504.3655 - val_loss: 1182.8536 - val_mean_squared_error: 1182.8536
Epoch 4/30
76651/76651 [==============================] - 23s 299us/step - loss: 14375.9580 - mean_squared_error: 14375.9580 - val_loss: 1262.4935 - val_mean_squared_error: 1262.4935
Epoch 5/30
76651/76651 [==============================] - 23s 299us/step - loss: 14263.2118 - mean_squared_error: 14263.2118 - val_loss: 1312.2103 - val_mean_squared_error: 1312.2103
Epoch 6/30
76651/76651 [===========

## RNN

In [44]:
from keras import layers

model = Sequential()
model.add(Embedding(max_words, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['mse', rmse])
history = model.fit(data_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(data_test, y_test))

Train on 76651 samples, validate on 71772 samples
Epoch 1/10
65280/76651 [========================>.....] - ETA: 48s - loss: -305.2624 - mean_squared_error: 13904.1293 - rmse: 25.2237

KeyboardInterrupt: ignored

In [0]:
from keras import backend
def rmse(y_true, y_pred):
	return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

## LSTM

In [40]:
model = Sequential()
model.add(Embedding(max_words, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['mse', rmse])
history = model.fit(data_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(data_test, y_test))
#validation_split=0.2

Train on 76651 samples, validate on 71772 samples
Epoch 1/10
76651/76651 [==============================] - 1574s 21ms/step - loss: -342.6725 - mean_squared_error: 15665.5694 - rmse: 25.2900 - val_loss: -238.3488 - val_mean_squared_error: 1354.1407 - val_rmse: 14.9506
Epoch 2/10
76651/76651 [==============================] - 1580s 21ms/step - loss: -403.0471 - mean_squared_error: 15665.1981 - rmse: 25.2815 - val_loss: -238.3488 - val_mean_squared_error: 1354.1407 - val_rmse: 14.9506
Epoch 3/10
76651/76651 [==============================] - 1597s 21ms/step - loss: -403.0471 - mean_squared_error: 15665.1979 - rmse: 25.2815 - val_loss: -238.3488 - val_mean_squared_error: 1354.1407 - val_rmse: 14.9506
Epoch 4/10
54784/76651 [====================>.........] - ETA: 6:16 - loss: -406.2701 - mean_squared_error: 18011.9057 - rmse: 25.4836

KeyboardInterrupt: ignored

## GRU

In [60]:
model = Sequential()
model.add(layers.Embedding(max_words, 128, input_length=1000))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

#model.summary()

model.compile(optimizer='rmsprop', loss='mse',metrics=['mse', rmse])

history = model.fit(data_train, y_train,
                    epochs=10,
                    batch_size=128,
                    validation_data=(data_test, y_test))

#history = model.fit((data_train, y_train),
#                              steps_per_epoch=500,
#                              epochs=20,
#                              batch_size=128,
#                              validation_data=(data_test, y_test))


Train on 76651 samples, validate on 71772 samples
Epoch 1/10
76651/76651 [==============================] - 493s 6ms/step - loss: 15188.9720 - mean_squared_error: 15188.9720 - rmse: 24.7576 - val_loss: 1048.4127 - val_mean_squared_error: 1048.4127 - val_rmse: 15.7861
Epoch 2/10
76651/76651 [==============================] - 495s 6ms/step - loss: 14862.2407 - mean_squared_error: 14862.2407 - rmse: 26.3465 - val_loss: 1036.8854 - val_mean_squared_error: 1036.8854 - val_rmse: 17.7897
Epoch 3/10
76651/76651 [==============================] - 493s 6ms/step - loss: 14659.9214 - mean_squared_error: 14659.9214 - rmse: 26.1800 - val_loss: 1032.4901 - val_mean_squared_error: 1032.4901 - val_rmse: 17.6127
Epoch 4/10
76651/76651 [==============================] - 496s 6ms/step - loss: 14499.2338 - mean_squared_error: 14499.2338 - rmse: 25.9838 - val_loss: 1028.7713 - val_mean_squared_error: 1028.7713 - val_rmse: 17.4337
Epoch 5/10
76651/76651 [==============================] - 493s 6ms/step - loss

In [64]:
history

## Criando uma camada de embedding de um modelo já treinado

Basta baixar no site: http://nilc.icmc.usp.br/embeddings e fazer os mesmos comandos.


Vantagens: o modelo treina mais rápido e aparente é mais estável (diminui o erro com o número de épocas).

In [50]:
glove = KeyedVectors.load_word2vec_format('glove_s50.txt')

ValueError: ignored

In [0]:
embed=glove.get_keras_embedding()

## Rede convolucional com o embedding Glove

In [0]:
model_glove= Sequential()
model_glove.add(embed)
model_glove.add(Convolution1D(32,7,activation='relu'))
model_glove.add(MaxPooling1D(5))
model_glove.add(Convolution1D(32,7,activation='relu'))
model_glove.add(GlobalMaxPooling1D())
model_glove.add(Dense(1, kernel_initializer='normal'))


model.summary()

In [0]:
model_glove.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])
history = model_glove.fit(data_train, y_train,
                    epochs=3,
                    batch_size=128,
                    validation_data=(data_test, y_test))

# Considerações 

Por enquanto estamos com resultados bons em todos modelos: RMSE entre 32 e 35 dias.

Para tentar diminuir isso eu sugiro os seguintes testes:

- Criar uma rede recorrente com LSTM ou GRU (já importei essas layers do Keras e é muito semelhante que construir uma rede convolucional).

- Criar redes recorrentes bidirecionais conforme o link: https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.3-advanced-usage-of-recurrent-neural-networks.ipynb

- Testar parâmetros diferentes no TF-IDF e Tokenizer, como montar as tabelas em sequência de 2 ou mais palavras, aumentar/diminuir features geradas e etc. É olhar no site para ver como faz, só alterar parâmetros das funções.

- Trocar modelo de embedding além do Glove 50. 

- Talvez excluir do treino os processos mais antigos, que em geral são meio que outliers.

- Por fim o cross-validation para comitê (média) das previsões.

Para esses modelos de redes, compensa usar nuvem para treino, pois demoram um bom tempo em treino.

In [0]:
#!cat /proc/cpuinfo